# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

weather_data_path = "weather_data.csv"

#read
weather_data = pd.read_csv(weather_data_path)

#df
df = pd.DataFrame(weather_data)

# Import API key
from config import gmaps_key

In [2]:
df.head()

,City,Country,lat,long,temp,humidity,cloudiness,wind speed
0,Maragogi,BR,-9.01,-35.22,79.826,73,16,4.69
1,San Andrés,CO,12.58,-81.70,82.400,88,75,6.20
2,Meulaboh,ID,4.14,96.13,81.320,77,92,1.20
3,Irving,US,32.81,-96.95,80.276,74,90,5.70
4,Stara Vyzhivka,UA,51.44,24.44,72.086,56,50,2.26


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
gmaps.configure(api_key=gmaps_key)

In [4]:
fig = gmaps.figure()

In [5]:
#test fig 
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#store humidity 
humidity = df["humidity"]
# humidity.astype({"humidity":float})
# humidity.dtypes

In [7]:
#store altitude and longitude together
locations = df[["lat","long"]]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.5)

In [9]:
# Add layer
fig.add_layer(heat_layer)

In [10]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# #find the values between 70 and 80
temp_clean = df[df['temp'].between(70, 80)]
temp_clean

,City,Country,lat,long,temp,humidity,cloudiness,wind speed
0,Maragogi,BR,-9.01,-35.22,79.826,73,16,4.69
4,Stara Vyzhivka,UA,51.44,24.44,72.086,56,50,2.26
9,Rikitea,PF,-23.12,-134.97,73.544,86,100,5.40
15,Albany,US,42.60,-73.97,71.096,74,98,0.95
18,Laredo,US,27.51,-99.51,79.754,78,1,5.10
...,...,...,...,...,...,...,...,...
538,Akonolinga,CM,3.77,12.25,70.304,96,100,0.47
543,Umred,IN,20.85,79.33,78.800,94,90,2.53
548,Borbon,PH,10.84,124.03,79.466,79,100,3.91
550,Binga,ZW,-17.62,27.34,79.304,16,0,2.08


In [12]:
#we want wind speed less than 10mph
wind_speed_ideal = temp_clean[temp_clean['wind speed']<10]
wind_speed_ideal

,City,Country,lat,long,temp,humidity,cloudiness,wind speed
0,Maragogi,BR,-9.01,-35.22,79.826,73,16,4.69
4,Stara Vyzhivka,UA,51.44,24.44,72.086,56,50,2.26
9,Rikitea,PF,-23.12,-134.97,73.544,86,100,5.40
15,Albany,US,42.60,-73.97,71.096,74,98,0.95
18,Laredo,US,27.51,-99.51,79.754,78,1,5.10
...,...,...,...,...,...,...,...,...
538,Akonolinga,CM,3.77,12.25,70.304,96,100,0.47
543,Umred,IN,20.85,79.33,78.800,94,90,2.53
548,Borbon,PH,10.84,124.03,79.466,79,100,3.91
550,Binga,ZW,-17.62,27.34,79.304,16,0,2.08


In [13]:
#we want 0 cloudiness 
no_cloudy = wind_speed_ideal[wind_speed_ideal['cloudiness']==0]
no_cloudy
ideal_weather = pd.DataFrame(no_cloudy)
ideal_weather

,City,Country,lat,long,temp,humidity,cloudiness,wind speed
65,Poum,NC,-20.23,164.02,70.862,73,0,6.35
93,Manakara,MG,-22.13,48.02,71.204,82,0,1.42
124,La Santisima Trinidad,BO,-14.83,-64.90,77.522,59,0,1.84
137,Malinovoye Ozero,RU,51.68,79.78,75.434,48,0,6.33
147,Mossamedes,AO,-15.20,12.15,77.000,65,0,6.20
276,Vangaindrano,MG,-23.35,47.60,71.384,80,0,2.06
304,Mandalgovi,MN,45.76,106.27,75.308,21,0,1.98
411,Gannan,CN,47.91,123.50,71.438,76,0,1.89
428,Henties Bay,NaN,-22.12,14.28,73.994,46,0,1.34
432,Likasi,CD,-10.98,26.73,79.160,17,0,4.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#store variable into hotel_df
hotel_df = ideal_weather
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,lat,long,temp,humidity,cloudiness,wind speed,Hotel Name
65,Poum,NC,-20.23,164.02,70.862,73,0,6.35,
93,Manakara,MG,-22.13,48.02,71.204,82,0,1.42,
124,La Santisima Trinidad,BO,-14.83,-64.90,77.522,59,0,1.84,
137,Malinovoye Ozero,RU,51.68,79.78,75.434,48,0,6.33,
147,Mossamedes,AO,-15.20,12.15,77.000,65,0,6.20,
276,Vangaindrano,MG,-23.35,47.60,71.384,80,0,2.06,
304,Mandalgovi,MN,45.76,106.27,75.308,21,0,1.98,
411,Gannan,CN,47.91,123.50,71.438,76,0,1.89,
428,Henties Bay,NaN,-22.12,14.28,73.994,46,0,1.34,
432,Likasi,CD,-10.98,26.73,79.160,17,0,4.21,


In [15]:
# find the closest restaurant of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": "5000",
    "type": "lodging",
    "keyword":"hotel",
    "key": gmaps_key}

In [17]:
for index, row in hotel_df.iterrows():

   # assemble url and make API request
    city = row["City"]
    
    #lat and long
    lat = row["lat"]
    long = row["long"]
    params["location"] = f"{lat},{long}"
    
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()

    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 65: Poum.
Missing field/result... skipping.
------------
Retrieving Results for Index 93: Manakara.
Closest hotel to Manakara is HOTEL H1 MANAKARA.
------------
Retrieving Results for Index 124: La Santisima Trinidad.
Closest hotel to La Santisima Trinidad is Hotel Río Ibare.
------------
Retrieving Results for Index 137: Malinovoye Ozero.
Missing field/result... skipping.
------------
Retrieving Results for Index 147: Mossamedes.
Closest hotel to Mossamedes is Hotel ibis Styles Iu Namibe.
------------
Retrieving Results for Index 276: Vangaindrano.
Closest hotel to Vangaindrano is Hotel Fanilo.
------------
Retrieving Results for Index 304: Mandalgovi.
Closest hotel to Mandalgovi is Mandal Hotel.
------------
Retrieving Results for Index 411: Gannan.
Closest hotel to Gannan is 甘南怡家快捷宾馆.
------------
Retrieving Results for Index 428: Henties Bay.
Closest hotel to Henties Bay is De Duine Hotel.
------------
Retrieving Results for Index 432: Likasi.
Closest h

In [18]:
hotel_df

,City,Country,lat,long,temp,humidity,cloudiness,wind speed,Hotel Name
65,Poum,NC,-20.23,164.02,70.862,73,0,6.35,
93,Manakara,MG,-22.13,48.02,71.204,82,0,1.42,HOTEL H1 MANAKARA
124,La Santisima Trinidad,BO,-14.83,-64.90,77.522,59,0,1.84,Hotel Río Ibare
137,Malinovoye Ozero,RU,51.68,79.78,75.434,48,0,6.33,
147,Mossamedes,AO,-15.20,12.15,77.000,65,0,6.20,Hotel ibis Styles Iu Namibe
276,Vangaindrano,MG,-23.35,47.60,71.384,80,0,2.06,Hotel Fanilo
304,Mandalgovi,MN,45.76,106.27,75.308,21,0,1.98,Mandal Hotel
411,Gannan,CN,47.91,123.50,71.438,76,0,1.89,甘南怡家快捷宾馆
428,Henties Bay,NaN,-22.12,14.28,73.994,46,0,1.34,De Duine Hotel
432,Likasi,CD,-10.98,26.73,79.160,17,0,4.21,Hotel De Likasi


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))